# Team 6 Project: MINJUSTICIA

In [64]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
pd.options.display.max_columns = 50

import plotly
import plotly.tools as tls
import chart_studio.plotly as py
from plotly.graph_objs import Figure, Data, Layout
import matplotlib.pyplot as plt
from plotly.graph_objs import *
%matplotlib inline
from pylab import rcParams
%matplotlib inline

path = "C:/Users/david/Desktop/ds4a-team6/retomintic/Data_UpdateJune13/reincidencia11junio2020_clean.csv"
df_mj = pd.read_csv(path, sep=",",encoding="utf-8")

# Processing

## 1. Data cleaning and tentative transformations

In [39]:
#Date variables are parsed to datetime
df_mj["FECHA_CAPTURA"] = pd.to_datetime(df_mj["FECHA_CAPTURA"])
df_mj["FECHA_INGRESO"] = pd.to_datetime(df_mj["FECHA_INGRESO"])
df_mj["FECHA_SALIDA"] = pd.to_datetime(df_mj["FECHA_SALIDA"])
#Month and year variables are defined
df_mj["MES_INGRESO_INT"]=df_mj["FECHA_INGRESO"].dt.strftime('%m')
df_mj["ANO_INGRESO_INT"]=df_mj["FECHA_INGRESO"].dt.strftime('%y')
#Calculations on how much time have the criminal being outside since its last stay in jail
for column in ['FECHA_INGRESO', 'FECHA_SALIDA', 'FECHA_CAPTURA']:
    df_mj = df_mj.sort_values(['INTERNOEN', column], ascending = False)
    
    df_mj['DIAS' + column[5:]] = -1*df_mj[column].diff().copy()

    df_mj.loc[(df_mj.INTERNOEN != df_mj.INTERNOEN.shift(1)) | (df_mj['DIAS' + column[5:]] == np.timedelta64(0)), 
              ['DIAS' + column[5:]]] = datetime.today() - df_mj[column]
    
#It seems that sometimes entering and gettint out is switched, that's why we computed in absolute values
df_mj['DIAS_CONDENA'] = abs(df_mj['FECHA_SALIDA'] - df_mj['FECHA_INGRESO'])
df_mj['DIAS_JUDICIALIZACION'] = df_mj['FECHA_INGRESO'] - df_mj['FECHA_CAPTURA']
df_mj['DIAS_LIBRE'] = df_mj['DIAS_INGRESO'] - df_mj['DIAS_CONDENA']
#The individual finishes its sentence but she's incarcelated inmediately for another crime
df_mj.loc[df_mj.DIAS_CAPTURA < np.timedelta64(0), 'DIAS_CAPTURA'] = np.timedelta64(0)
df_mj.loc[df_mj.DIAS_INGRESO < np.timedelta64(0), 'DIAS_INGRESO'] = np.timedelta64(0)
df_mj.loc[df_mj.DIAS_LIBRE < np.timedelta64(0), 'DIAS_LIBRE'] = np.timedelta64(0)
#The individual is still on jail
df_mj.loc[df_mj['DIAS_LIBRE'].isnull(), 'DIAS_LIBRE'] = np.timedelta64(0)

#Timedelta variables are parsed to integer variables
df_mj[['DIAS_CONDENA', 'DIAS_LIBRE', 'DIAS_INGRESO']] = df_mj[['DIAS_CONDENA', 'DIAS_LIBRE', 'DIAS_INGRESO']].apply(lambda x: x.dt.days, axis = 0)

#Find the last date the criminal went out the jail, so that these observations are marked as censored
last_df = df_mj[['INTERNOEN', 'FECHA_INGRESO']].groupby('INTERNOEN').apply(lambda x: x.sort_values('FECHA_INGRESO', ascending = False).head(1)).reset_index(drop = True)
#Censored
last_df['CENSURADO_LIBRES'] = 0
df_mj = df_mj.merge(last_df, on = ['INTERNOEN', 'FECHA_INGRESO'], how = 'left')
#Event
df_mj.loc[df_mj['CENSURADO_LIBRES'].isnull(), 'CENSURADO_LIBRES'] = 1
#All criminals that haven't got out of jail yet have zero days out and they are not censored.
df_mj.loc[df_mj['FECHA_SALIDA'].isnull(), 'CENSURADO_LIBRES'] = 1
#Turned censored variables to integers instead of float
df_mj['CENSURADO_LIBRES'] = df_mj['CENSURADO_LIBRES'].astype('int64')

#We create a variable to count the amount of times the individual re-entered in jail
df_mj = df_mj.merge(df_mj.drop_duplicates(['INTERNOEN', 'FECHA_INGRESO']).groupby(['INTERNOEN']).size().reset_index(name = 'NUMERO_REINCIDENCIAS'), on = 'INTERNOEN', how = 'left')
#This is the average amount of crimes the ciminals had before he committed its last crime
index_not_last = df_mj.groupby(['INTERNOEN'])['FECHA_INGRESO'].apply(lambda x: x != x.max())
df_mj = df_mj.merge(df_mj[index_not_last].groupby(['INTERNOEN', 'FECHA_INGRESO']).size().reset_index(name = 'CRIMENES').groupby('INTERNOEN')['CRIMENES'].mean().reset_index(name = 'CRIMENES_PROMEDIO'), on = 'INTERNOEN', how = 'left')
df_mj['CRIMENES_PROMEDIO'].fillna(0, inplace = True)

#We dropped SITUACION_JURIDICA and REINCIDENTE as both columns are constants
df_mj = df_mj.drop(columns = ['SITUACION_JURIDICA', 'REINCIDENTE'])

In [59]:
df_mj[~index_not_last].drop_values(['INTERNOEN', ])

,INTERNOEN,DELITO,TITULO_DELITO,SUBTITULO_DELITO,TENTATIVA,AGRAVADO,CALIFICADO,FECHA_INGRESO,FECHA_SALIDA,FECHA_CAPTURA,ANO_NACIMIENTO,EDAD,GENERO,ESTADO_CIVIL,PAIS_INTERNO,DEPARTAMENTO,CIUDAD,ESTADO_INGRESO,ACTIVIDADES_TRABAJO,ACTIVIDADES_ESTUDIO,ACTIVIDADES_ENSEÑANZA,NIVEL_EDUCATIVO,HIJOS_MENORES,CONDIC_EXPECIONAL,CODIGO_ESTABLECIMIENTO,ESTABLECIMIENTO,DEPTO_ESTABLECIMIENTO,MPIO_ESTABLECIMIENTO,REGIONAL,ESTADO,MES_INGRESO_INT,ANO_INGRESO_INT,DIAS_INGRESO,DIAS_SALIDA,DIAS_CAPTURA,DIAS_CONDENA,DIAS_JUDICIALIZACION,DIAS_LIBRE,CENSURADO_LIBRES,NUMERO_REINCIDENCIAS,CRIMENES_PROMEDIO
0,FFFFB0916FD16C7AF652FF211662005C40F70E33,FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO ...,CONTRA LA SEGURIDAD PUBLICA,DE LOS DELITOS DE PELIGRO COMUN O QUE PUEDEN O...,N,N,N,2016-05-02,NaT,2016-02-02,1990,29,MASCULINO,UNION LIBRE,REPUBLICA DE COLOMBIA,RISARALDA,PEREIRA,Intramuros,SI,NO,NO,CICLO IV,SI,NaN,502,EPMSC MEDELLIN,ANTIOQUIA,MEDELLIN,NOROESTE,Activo,05,16,1523,-9223372036854775808,1613 days 16:52:51.589919,NaN,90 days,0,1,1,0.0
1,FFFFB0916FD16C7AF652FF211662005C40F70E33,HOMICIDIO,CONTRA LA VIDA Y LA INTEGRIDAD PERSONAL,DE LA MANIPULACION GENETICA,N,S,N,2016-05-02,NaT,2016-02-02,1990,29,MASCULINO,UNION LIBRE,REPUBLICA DE COLOMBIA,RISARALDA,PEREIRA,Intramuros,SI,NO,NO,CICLO IV,SI,NaN,502,EPMSC MEDELLIN,ANTIOQUIA,MEDELLIN,NOROESTE,Activo,05,16,1523,-9223372036854775808,1613 days 16:52:51.589919,NaN,90 days,0,1,1,0.0
2,FFFFB0916FD16C7AF652FF211662005C40F70E33,HOMICIDIO,CONTRA LA VIDA Y LA INTEGRIDAD PERSONAL,DE LA MANIPULACION GENETICA,S,N,N,2016-05-02,NaT,2016-02-02,1990,29,MASCULINO,UNION LIBRE,REPUBLICA DE COLOMBIA,RISARALDA,PEREIRA,Intramuros,SI,NO,NO,CICLO IV,SI,NaN,502,EPMSC MEDELLIN,ANTIOQUIA,MEDELLIN,NOROESTE,Activo,05,16,1523,-9223372036854775808,1613 days 16:52:51.589919,NaN,90 days,0,1,1,0.0
3,FFFF73D0B7088A0EF0A4B7F3CC26F8CF672C6EC1,HURTO,CONTRA EL PATRIMONIO ECONOMICO,DE LA ESTAFA,N,N,S,2016-05-04,2016-09-06,2014-11-06,1986,33,MASCULINO,NaN,REPUBLICA DE COLOMBIA,BOGOTA DISTRITO CAPITAL,BOGOTA DISTRITO CAPITAL,Prision Domiciliaria,NO,NO,NO,CICLO IV,SI,NaN,113,COMPLEJO CARCELARIO Y PENITENCIARIO METROPOLIT...,BOGOTA D.C.,BOGOTA D.C.,CENTRAL,Baja,05,16,1521,120675171134042000,2066 days 16:52:51.589919,125.0,545 days,1396,0,1,0.0
4,FFFE667F85CF452DB70B955609103CB72ECA3EEB,FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO ...,CONTRA LA SEGURIDAD PUBLICA,DE LOS DELITOS DE PELIGRO COMUN O QUE PUEDEN O...,N,N,N,2020-01-14,NaT,2019-12-16,1984,36,MASCULINO,UNION LIBRE,REPUBLICA DE COLOMBIA,NORTE DE SANTANDER,CUCUTA-NORTE DE SANTANDER,Intramuros,SI,NO,NO,CICLO II,SI,NaN,4222,COMPLEJO CARCELARIO Y PENITENCIARIO METROPOLIT...,NORTE DE SANTANDER,CUCUTA,ORIENTE,Activo,01,20,171,-9223372036854775808,200 days 16:52:51.589919,NaN,29 days,0,1,1,0.0
